In [36]:
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px
import os

import matplotlib.pyplot as plt
%matplotlib inline

# Exploration

The purpose of this notebook is to get familiar with the relevant data sets that are used during the course. You will see examples of how to work with numpy, pandas and plotting libraries. 

## Detailed population data

INSEE provides detailed population information per municipality as open data:

- Go to https://www.insee.fr/fr/statistiques/6544333
- Download "Individus localisés au canton-ou-ville" in CSV format
- Put the downloaded zip file into the folder that contains this notebook
- Unpack the zip file so that the CSV file is located next to this notebook
- Alternatively call the following cell if you are using Linux

In [37]:
if not os.path.exists("FD_INDCVI_2019.csv"):
    !wget https://www.insee.fr/fr/statistiques/fichier/6544333/RP2019_INDCVI_csv.zip
    !unzip RP2019_INDCVI_csv.zip

Next, load a chunk of the data to see what is contained in the file:

In [38]:
df_census = pd.read_csv("FD_INDCVI_2019.csv", sep = ";", nrows = 10)
df_census.head()

,CANTVILLE,NUMMI,ACHLR,AEMMR,AGED,AGER20,AGEREV,AGEREVQ,ANAI,ANEMR,...,TP,TRANS,TRIRIS,TYPC,TYPFC,TYPL,TYPMC,TYPMR,VOIT,WC
0,101,1,4,9,68,79,67,65,1949,1,...,Z,Z,ZZZZZZ,3,Z,2,1,11,1,Z
1,101,2,5,9,33,39,32,30,1983,0,...,1,5,ZZZZZZ,2,2,1,4,41,2,Z
2,101,2,5,9,1,2,0,0,2015,0,...,Z,Z,ZZZZZZ,2,2,1,4,41,2,Z
3,101,2,5,9,30,29,29,25,1986,0,...,1,5,ZZZZZZ,2,2,1,4,41,2,Z
4,101,3,4,9,72,79,71,70,1948,2,...,Z,Z,ZZZZZZ,3,Z,2,1,12,0,Z


You will find information on the variables in the dataset by clicking on "Dictionnaire des variables" in the link above and downloading the corresponding PDF.

**Task**:  For our first analysis, load the following columns. For performance reasons, it makes sense, to define a data type for each column:
- Detailed age (by year) as `int`
- Socioprofessional category (Catégorie socioprofessionnelle en 8 postes) as `int`

Additionally, load the following columns:
- `IRIS` is an identifier for the location of the observation in France, load it as `str`
- `IPONDI` is a weight of each observation, load it as `float`

In [39]:
columns = {
    "IRIS": str,
    "IPONDI": float,
    
    # Insert the other columns here
    # ...

    "AGED": int,
    "CS1": int
}

df_census = pd.read_csv("FD_INDCVI_2019.csv", sep = ";", dtype = columns, usecols = columns.keys())
df_census.head()

KeyboardInterrupt: 

**Task**:  It is always better to work with a cleaned data set, let's clean up the column names:
- The age column to `age`
- The socioprofessional category column to `csp`

In [ ]:
df_census = df_census.rename(columns = {
    "IRIS": "iris_id",
    "IPONDI": "weight",

    # Insert code here
    # ...

    "AGED": "age",
    "CS1": "csp"
})

df_census.head()

The data set contains the official open census data from the French statistical office INSEE. Let's aggregate the data to obtain a data frame that gives us the number of persons at a certain age:

In [ ]:
df_age = df_census.groupby("age").size().reset_index(name = "count")
df_age.head()

And plot this information using plotly:

In [ ]:
px.bar(df_age, x = "age", y = "count")

Is this information correct? Write the code to calculate total number of observations in the data set:

In [ ]:
df_census.size

Compare this value with information from other sources like Wikipedia? Do we see a difference? Why?

**Task**: Write the code to calculate the correct number of French inhabitants:

In [ ]:
# Insert code here
# ...

df_census["weight"].sum()

**Task:** Show a bar plot of both the count of *observations* at a specific age and the number of *persons*.

Hints: 
- You will need another aggregator function than `size` (used before) in your `groupby` statement
- You will need to `merge` the existing `df_age` data frame and a new one that you create
- For the y-axis, you may pass a list of columns to plotly
- Try to use the `barmode = "group"` argument for plotly

In [ ]:
# Insert code here
# ...

df_age_persons = df_census.groupby("age")["weight"].sum().reset_index()
df_age = pd.merge(df_age, df_age_persons, on = "age")

px.bar(df_age, x = "age", y = ["count", "weight"], barmode = "group")

**Task:** On average, how many persons are represented by one observation in the census data?

In [ ]:
# Insert code here
# ...

df_census["weight"].mean()

Let's explore the data a bit further. 

**Task:** Show the number of persons for each socioprofessional category in a plot.

Bonus: Instead of showing only CSP identifiers, can you show the name of the CSPs?

Remember, the socioprofessional category is a classification of persons in France according to their job status:
https://www.insee.fr/fr/metadonnees/pcs2003/categorieSocioprofessionnelleAgregee/1?champRecherche=true

In [ ]:
# Insert code here
# ...

df_csp = df_census.groupby("csp")["weight"].sum().reset_index()
df_csp = df_csp.replace({
    1: "Agriculteurs",
    2: "Artisans",
    3: "Cadres",
    4: "Intermédiaires",
    5: "Employés",
    6: "Ouvriers",
    7: "Retraités",
    8: "Autres"
})

px.bar(df_csp, x = "csp", y = "weight")

**Task:** Show a bar plot with one age distribution per CSP in different colors and use it to compare the age distribution of at least three CSP.

Hint:
- You will need to aggregate over two columns this time.

In [ ]:
# Insert code here
# ...

df_age = df_census.groupby(["age", "csp"])["weight"].sum().reset_index()
df_age = df_age[df_age["csp"].isin([1, 3, 5])]
df_age["csp"] = df_age["csp"].astype(str)

px.bar(df_age, x = "age", y = "weight", color = "csp", barmode = "group")

The previous analysis were performed in absolute terms. Let's pass on to a relative analysis. We want to know which percentage of people belonging to a certain CSP has a certain age. So the bars should sum up to one per CSP.

**Task:** Set up a plot where two or three CSP are shown with their relative age distribution, summing up to one for each CSP.

Hints:
- Proceed as in the previous task, but perform a second aggregation by CSP.
- Via `merge`, append another column to the two-variable data set that describes this total
- Then, divide the absolute value by the group total

In [ ]:
# Insert code here
# ...

df_age = df_census.groupby(["age", "csp"])["weight"].sum().reset_index()
df_total = df_census.groupby("csp")["weight"].sum().reset_index().rename(columns = { "weight": "total" })

df_age = pd.merge(df_age, df_total, on = "csp")
df_age["share"] = df_age["weight"] / df_age["total"]

df_age = df_age[df_age["csp"].isin([1, 3, 5])]
df_age["csp"] = df_age["csp"].astype(str)

px.bar(df_age, x = "age", y = "share", color = "csp", barmode = "group")

## Spatial data

So far, we have only performed analysis over the whole French population. The data set contains a column called `IRIS`. This is a statistical zoning system that covers France. Each zone in that system has a unique identifier. It is constructed as follows:

- `[2]` digits are the department identifier
- `[3]` following digits describe the municipality
- `[4]` following digits describe the IRIS (sub-municipality zoning)

For instance, the 14e arrondissement in Paris has the `75` as the department identifier `75`, followed by `114` indicating the arrondissement. After, there are four digits that describe smaller zones within the arrondissement, for instance:

`[75][115][0001]`

**Task:** For convience, let's create additional columns that indicate the department and the municipality of an observation:
- `department_id`: The first two digits of `iris_id`
- `municipality_id`: The five first digits of `iris_id`

In [ ]:
### Insert code here
# ...

df_census["department_id"] = df_census["iris_id"].str[:2]
df_census["municipality_id"] = df_census["iris_id"].str[:5]

**Task:** Find out which are the 10 departments with the highest number of inhabitants. Which are the 10 least inhabitated ones?

In [ ]:
### Insert code here
# ...

df_census.groupby("department_id")["weight"].sum().reset_index().sort_values(by = "weight", ascending = True)

Do you observe anything special?

**Task:** Let's repeat the exercise by identifying the top 10 and bottom 10 municipalities:

In [ ]:
### Insert code here
# ...

df_census.groupby("municipality_id")["weight"].sum().reset_index().sort_values(by = "weight", ascending = False)

## Mapping

Looking at spatial data works best when using maps. The IRIS system is not only a system of identifiers, but there is also geographic shape data attached to it. The data is provided by IGN (Institut Géographique National).

- Download the data from https://geoservices.ign.fr/contoursiris
- Make sure to download the 2021 edition which is compatible with our 2019 census data
- Unpack the 7z file in the current folder
- The relevant files are located in `CONTOURS*/1_DONNES/*LAMB93*/` (make sure about the last `LAMB93` part)
- Copy the files prefixed with `CONTOURS-IRIS.*` to the folder where this notebook is located

Let's load the data using `geopandas`:

In [ ]:
df_iris = gpd.read_file("CONTOURS-IRIS.shp")
df_iris.head()

As before, let's clean up the data set. We will need the following columns with the following readable names:
- `INSEE_COM`: `municipality_id`
- `CODE_IRIS`: `iris_id`
- `geometry`

**Task:** Set up the data set accordingly.

In [ ]:
### Insert code here
# ...

df_iris = df_iris[["INSEE_COM", "CODE_IRIS", "geometry"]].rename(columns = {
    "INSEE_COM": "municipality_id", "CODE_IRIS": "iris_id"
})

**Task**: Calculate how many IRIS exist in France and how many municipalities are there:

In [ ]:
len(df_iris["municipality_id"].unique()), len(df_iris["iris_id"].unique())

You can try plotting all IRIS or all municipalities, but this will usually take a while with the standard Python tools. Let's plot only Paris:

In [ ]:
df_iris[df_iris["municipality_id"].str.startswith("75")].plot()

**Task**: Only the spatial shapes are not really useful. We should attach some data to it. To simplify our life, let's create a data frame based on `df_iris` that only contains the municipality shapes `df_municipalities`.

Hint: Check the `dissolve` method in `geopandas`.

In [ ]:
### Insert code here
# ...

# df_municipalities = 

df_municipalities = df_iris.dissolve("municipality_id").reset_index()

**Task**: Plot all municipalities in the Essonne departmennt (91).

In [ ]:
### Insert code here
# ...

df_municipalities[df_municipalities["municipality_id"].str.startswith("91")].plot()

The following list contains all departments in the Île-de-France region:

In [ ]:
idf_departments = ["75", "92", "93", "94", "95", "77", "91", "78"]

**Task**: Show all municipalities in the Île-de-France region.

In [ ]:
### Insert code here
# ...

df_municipalities["department_id"] = df_municipalities["municipality_id"].str[:2]
df_municipalities[df_municipalities["department_id"].isin(idf_departments)].plot()

**Task**: Now we are ready to cross some information with the spatial data set:
- Prepare a data set that contains the number of inhabitants per municipality (`municipality_id`, `inhabitants`)
- Perform a merge between your municipality data frame and the inhabitant data frame
- Provide the inhabitants column in the `plot` method

In [ ]:
### Insert code here
# ...

df_inhabitants = df_census.groupby("municipality_id")["weight"].sum().reset_index(name = "inhabitants")
pd.merge(df_municipalities, df_inhabitants).plot("inhabitants")

**Task**: Plot a population map of Île-de-France with a legend (using `legend = True`)

In [ ]:
### Insert code here
# ...

df_inhabitants = df_census.groupby("municipality_id")["weight"].sum().reset_index(name = "inhabitants")
pd.merge(df_municipalities[df_municipalities["department_id"].isin(idf_departments)], df_inhabitants).plot("inhabitants")

Do you observe anything specific?

## Aggregated population data

To solve the issue, INSEE provides aggregated census data sets with less attributes but higher spatial availability. We will make use of a data set that indicates the total population and population per CSP over 15 years for every municipality in France:

- The data is available at https://www.insee.fr/fr/statistiques/6543200
- Download "Population en 2019 - IRIS - France hors Mayotte" in CSV format
- Information on the variables is avaialble in "Dictionnaire des variables"

**Task**: Load the data set have a first look
- Only load a couple of lines (`nrows=20`) to be sure that you don't exceed your memory
- Look at the first few lines and check the explanation of the variables online
- How can you obtain the population total per municipality from this data set?
- How can you obtain the number of persons per CSP from this data set?

In [ ]:
### Insert code here
# ...

df_population = pd.read_csv("base-ic-evol-struct-pop-2019.CSV", sep = ";", nrows = 20)
df_population

**Task**: Transform the data set such that you have each municipality together with the population total and the total of each CSP:

In [ ]:
pd.DataFrame({ "municipality_id": [], "population": [], "csp_1": [], "csp_2": [], "csp_3": [], "csp_...": [] })

Hint: The data set is given per IRIS.

In [ ]:
### Insert code here
# ...

# df_population = ...

df_population = pd.read_csv("base-ic-evol-struct-pop-2019.CSV", sep = ";", dtype = {
    "COM": "str"
}, usecols = ["COM", "P19_POP"] + ["C19_POP15P_CS{}".format(k) for k in range(1,9)])

for k in range(1, 9):
    df_population["csp_{}".format(k)] = df_population["C19_POP15P_CS{}".format(k)]
    
df_population = df_population[["COM", "P19_POP"] + ["csp_{}".format(k) for k in range(1,9)]]
df_population.columns = ["municipality_id", "population"] + ["csp_{}".format(k) for k in range(1,9)]
df_population = df_population.groupby("municipality_id").sum().reset_index()

**Task**: Repeat the task from above, create a map of the population in Île-de-France, but with the new data set.

In [ ]:
### Insert code here
# ...

pd.merge(df_municipalities[df_municipalities["department_id"].isin(idf_departments)], df_population).plot("population")

Save the cleaned population data, because we will need it again in a later exercise:

In [ ]:
df_population.to_parquet("population.parquet")

Let's do the same with the municipalities spatial data set:

In [ ]:
df_municipalities.to_parquet("municipalities.parquet")

For mapping, Python can be useful to make a first draft, but there are more elaborate tools available. 

**Task**: Create a data frame in which the municipality data has been merged with the the population data set, i.e., we want all columns from the population data set and additionally the `geometry` column. Filter for all municipalities in Île-de-France. Save this data frame in GeoPackage format:

In [ ]:
### Insert code here
# ...

# df_export = ...

df_export = pd.merge(df_municipalities[df_municipalities["department_id"].isin(
    idf_departments
)], df_population, on = "municipality_id")

In [ ]:
df_export.to_file("export.gpkg")

**Exercise**: Explore the exported data using **QGIS**

![](qgis.png)

## Employment

In a later exercise, we will also need information on employment. Employment data per municipality is avaialble as open data from Urssaf.

- Download the data from https://open.urssaf.fr/explore/dataset/etablissements-et-effectifs-salaries-au-niveau-commune-x-ape-last/information/
- Go to "Export" and export the data as CSV

**Task**: As before, explore the data by first loading a few columns and understanding the content.

Hint: To get a better overview of the available columns, try `df.columns`

In [ ]:
### Insert code here
# ...

df_employment = pd.read_csv("etablissements-et-effectifs-salaries-au-niveau-commune-x-ape-last.csv", nrows = 20, sep = ";")
df_employment.head()
df_employment.columns

**Task**: Clean the data set such that you have a column indicating the municipality identifier and the number of emloyees in 2019 in that zone.

Hint: The data frame is disaggregated over various economic sectors (NAF code), but we want the total!

In [ ]:
pd.DataFrame({ "municipality_id": [], "emloyment": [] })

In [ ]:
### Insert code here
# ...

# df_employment = ...

df_employment = pd.read_csv("etablissements-et-effectifs-salaries-au-niveau-commune-x-ape-last.csv", sep = ";", 
    usecols = ["code_commune", "effectifs_salaries_2019"], dtype = { "code_commune": str })

df_employment = df_employment.rename(columns = {
    "code_commune": "municipality_id",
    "effectifs_salaries_2019": "employment"
})

df_employment = df_employment.groupby("municipality_id").sum().reset_index()

df_employment.head()

**Task**: Plot a map of the number of employees in a department of your choice

In [ ]:
### Insert code here
# ...

pd.merge(df_municipalities[df_municipalities["department_id"] == "63"], df_employment).plot("employment")

Let's save this data for later:

In [ ]:
df_employment.to_parquet("employment.parquet")

## Commuting data

Finally, we will have a look at a more complex data set: commuting data. This data set is also available from INSEE and describes how many people living in a specific municipality in France to to any other municipality for work. This data set is known as *MOBPRO*.

- Download the data from https://www.insee.fr/fr/statistiques/6456056
- Download the data in CSV format

**Task**: Load the data set with the following columns:
- `COMMUNE` : `str`
- `ARM` : `str`
- `DCLT`: `str`
- `IPONDI`: `float`
- `TRANS`: `int`

In [ ]:
### Insert code here
# ...

# df_commutes = ...

columns = { "COMMUNE": str, "ARM": str, "DCLT": str, "IPONDI": float, "TRANS": int }
df_commutes = pd.read_csv("RP2019_MOBPRO_csv/FD_MOBPRO_2019.csv", dtype = columns, usecols = columns.keys(), sep = ";")

The MOBPRO data set is a bit particular with respect to the spatial identifiers. In fact, `DCLT` describes the destination of a commuters as a municipality identifier. In principle, this is also the case for `COMMUNE` which describes the origin. However, Paris, for instance, is encoded as `75056`, but the actual "municipality" (or arrondissement) is contained in `ARM`. If one knows this, there is an easy fix:

In [ ]:
f = df_commutes["ARM"] != "ZZZZZ"
df_commutes.loc[f, "COMMUNE"] = df_commutes.loc[f, "ARM"]

**Task**: Reformat the data frame so that we have the following format.

In [ ]:
pd.DataFrame({ "origin_id": [], "destination_id": [], "weight": [], "transport_mode": [] })

In [ ]:
### Insert code here
# ...

df_commutes = df_commutes.rename(columns = {
    "COMMUNE": "origin_id", "DCLT": "destination_id", "IPONDI": "weight", "TRANS": "transport_mode"
})[["origin_id", "destination_id", "weight", "transport_mode"]]

**Task**: Plot a map showing how many people commute from Alfortville (94002) in the south of Paris to any other municipality in Île-de-France **by car**.

In [ ]:
### Insert code here
# ...

df = df_commutes[df_commutes["transport_mode"] == 5]
df = df[df_commutes["origin_id"] == "94002"]
df = df.groupby("destination_id")["weight"].sum().reset_index()
df = df.rename(columns = { "destination_id": "municipality_id" })

pd.merge(df_municipalities[df_municipalities["department_id"].isin(idf_departments)], df).plot("weight")

**Task**: Plot the same map but for commutes by **public transport**. What do you notice?

In [ ]:
### Insert code here
# ...

df = df_commutes[df_commutes["transport_mode"] == 6]
df = df[df["origin_id"] == "94002"]
df = df.groupby("destination_id")["weight"].sum().reset_index()
df = df.rename(columns = { "destination_id": "municipality_id" })

pd.merge(df_municipalities[df_municipalities["department_id"].isin(idf_departments)], df).plot("weight")

**Task:** Aggregate the commuting data set further by removing the `transport_mode` column such that we only have the bare commuting flows as a weight between two municipalities. Then, save the data set as `commutes.parquet`, we will need it later on!

In [ ]:
### Insert code here
# ...

df_commutes = df_commutes.groupby(["origin_id", "destination_id"])["weight"].sum().reset_index()
df_commutes.to_parquet("commutes.parquet")

**Congratulations!** You can now solve Exercise 1 of the course project.